# Algoritmo de Asignación de Tareas

In [176]:
import numpy as np
import pandas as pd
from scipy.optimize import linear_sum_assignment

In [177]:
def generar_costos_aleatorios(num_trabajadores, num_tareas):
    costos = np.random.randint(1, 101, size=(num_trabajadores, num_tareas))
    columnas = [f"Tarea {i+1}" for i in range(num_tareas)]
    filas = [f"Trabajador {i+1}" for i in range(num_trabajadores)]
    df = pd.DataFrame(costos, index=filas, columns=columnas)
    print("\nMatriz de costos generada aleatoriamente:")
    print(df)
    return costos

In [178]:
def rellenar_matriz(costos):
    filas, columnas = costos.shape
    if filas == columnas:
        return costos
    if filas > columnas:
        padding = np.full((filas, filas - columnas), 9999)
        costos = np.hstack((costos, padding))
    else:
        padding = np.full((columnas - filas, columnas), 9999)
        costos = np.vstack((costos, padding))
    return costos

In [179]:
def algoritmo_asignacion(costos):
    fila_indices, columna_indices = linear_sum_assignment(costos)
    asignaciones = list(zip(fila_indices, columna_indices))
    costo_total = costos[fila_indices, columna_indices].sum()
    return asignaciones, costo_total

In [ ]:
def hungarian_algorithm(cost_matrix):
    cost_matrix = cost_matrix.copy()
    n = cost_matrix.shape[0]

    print("Paso 1: reducción por filas")
    for i in range(n):
        cost_matrix[i] -= cost_matrix[i].min()
        print(cost_matrix[i])

    print("\nPaso 2: reducción por columnas")
    for j in range(n):
        cost_matrix[:, j] -= cost_matrix[:, j].min()
        print(cost_matrix[:, j])

    # encuentra ceros únicos
    def find_zeroes(m):
        return [(i, j) for i in range(n) for j in range(n) if m[i][j] == 0]

    # encuentra todas las coordenadas donde hay ceros en la matriz
    # este asigna sin que dos estén en la misma fila o columna
    def cover_zeroes(m):
        row_covered = [False] * n
        col_covered = [False] * n
        zero_locations = find_zeroes(m)
        print(f"\nCeros encontrados: {zero_locations}")
        marked_zeros = []

        for r, c in zero_locations:
            if not row_covered[r] and not col_covered[c]:
                marked_zeros.append((r, c))
                row_covered[r] = True
                col_covered[c] = True
                # print(f" fila - {r}: {row_covered}")
                # print(f" colu - {c}: {col_covered}")

        return marked_zeros

    def min_uncovered(m, row_cov, col_cov):
        return min(
            [m[i][j] for i in range(n) for j in range(n)
             if not row_cov[i] and not col_cov[j]]
        )

    while True:
        marked = cover_zeroes(cost_matrix)
        print(f"Combinacion: {marked}")
        # si trabajadores == tareas
        if len(marked) == n:
            return marked  # lista de asignaciones (trabajador, tarea)
        else:
            print(f"  Opcion no cubre asignacion de todas las tareas o trabajadores")

        # si trabajadores != tareas
        row_cov = [False] * n
        col_cov = [False] * n
        for r, c in marked:
            col_cov[c] = True

        # por si hay ceros sin cubrir
        while True:
            zero_found = False
            for i in range(n):
                for j in range(n):
                    if cost_matrix[i][j] == 0 and not row_cov[i] and not col_cov[j]:
                        row_cov[i] = True
                        col_cov[j] = False
                        zero_found = True
            if not zero_found:
                break
        
        # se crean nuevos ceros si comb actual no es la correcta
        min_val = min_uncovered(cost_matrix, row_cov, col_cov)
        for i in range(n):
            for j in range(n):
                if not row_cov[i] and not col_cov[j]:
                    # resta ese valor de las posiciones no cubiertas (crea más ceros)
                    cost_matrix[i][j] -= min_val
                if row_cov[i] and col_cov[j]:
                    # suma ese valor a las posiciones doblemente cubiertas (para mantener el equilibrio)
                    cost_matrix[i][j] += min_val
        
        print("\nMatriz ajustada:")
        print(cost_matrix)
        print("Ceros después del ajuste:", find_zeroes(cost_matrix))

In [206]:
def algoritmo_asignacion_manual(costos):
    asignaciones = hungarian_algorithm(costos)
    costo_total = sum(costos[i][j] for i, j in asignaciones)
    return asignaciones, costo_total

In [225]:
print("Algoritmo de asignación de tareas\n")
num_trabajadores = int(input("Número de trabajadores: "))
num_tareas = int(input("Número de tareas: "))

costos = generar_costos_aleatorios(num_trabajadores, num_tareas)
matriz_completa = rellenar_matriz(costos)

Algoritmo de asignación de tareas


Matriz de costos generada aleatoriamente:
              Tarea 1  Tarea 2  Tarea 3  Tarea 4  Tarea 5
Trabajador 1       68       79       59       41       63
Trabajador 2       36       99       99       27       34
Trabajador 3        8       72       10       11       50
Trabajador 4       53       44       10       56       81
Trabajador 5       37       31       80       90       22


### Elegir el método

In [226]:
opt = int(input("Ingrese la opcion que quiere intentar:\n 1. Algoritmo de asignacion de Python\n 2. Algoritmo de asignacion manual"))
if opt == 2:
    asignaciones, costo_total = algoritmo_asignacion_manual(matriz_completa)
else:
    asignaciones, costo_total = algoritmo_asignacion(matriz_completa)

Paso 1: reducción por filas
[27 38 18  0 22]
[ 9 72 72  0  7]
[ 0 64  2  3 42]
[43 34  0 46 71]
[15  9 58 68  0]

Paso 2: reducción por columnas
[27  9  0 43 15]
[29 63 55 25  0]
[18 72  2  0 58]
[ 0  0  3 46 68]
[22  7 42 71  0]

Ceros encontrados: [(0, 3), (1, 3), (2, 0), (3, 2), (4, 1), (4, 4)]
Combinacion: [(0, 3), (2, 0), (3, 2), (4, 1)]
  Opcion no cubre asignacion de todas las tareas o trabajadores

Matriz ajustada:
[[27 29 18  0 15]
 [ 9 63 72  0  0]
 [ 0 55  2  3 35]
 [43 25  0 46 64]
 [22  7 65 75  0]]
Ceros después del ajuste: [(0, 3), (1, 3), (1, 4), (2, 0), (3, 2), (4, 4)]

Ceros encontrados: [(0, 3), (1, 3), (1, 4), (2, 0), (3, 2), (4, 4)]
Combinacion: [(0, 3), (1, 4), (2, 0), (3, 2)]
  Opcion no cubre asignacion de todas las tareas o trabajadores

Matriz ajustada:
[[27 22 18  0 15]
 [ 9 56 72  0  0]
 [ 0 48  2  3 35]
 [43 18  0 46 64]
 [22  0 65 75  0]]
Ceros después del ajuste: [(0, 3), (1, 3), (1, 4), (2, 0), (3, 2), (4, 1), (4, 4)]

Ceros encontrados: [(0, 3), (1, 3),

In [208]:
print("\nAsignaciones óptimas:")
for trabajador, tarea in asignaciones:
    if trabajador < num_trabajadores and tarea < num_tareas:
        print(f"Trabajador {trabajador + 1} → Tarea {tarea + 1} (Costo: {costos[trabajador][tarea]})")
print(f"\nCosto total mínimo: {costo_total}")


Asignaciones óptimas:
Trabajador 1 → Tarea 3 (Costo: 17)
Trabajador 2 → Tarea 2 (Costo: 3)
Trabajador 3 → Tarea 4 (Costo: 10)
Trabajador 4 → Tarea 1 (Costo: 6)
Trabajador 6 → Tarea 5 (Costo: 9)

Costo total mínimo: 20043
